In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

data_path = Path("../data/Unified Dataset/unified_dataset_no_hypothyroid.csv")
df = pd.read_csv(data_path)

df.shape, df.head()

C:\Users\Click\AppData\Local\Temp\ipykernel_12708\3779319783.py:6: DtypeWarning: Columns (10,11,12,13,14,15,18,19,21,28,29,30,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


((221863, 37),
     patient_id     source   age sex country ethnicity family_history  \
 0  840801013.0  thyroidDF  29.0   F     NaN       NaN            NaN   
 1  840801014.0  thyroidDF  29.0   F     NaN       NaN            NaN   
 2  840801042.0  thyroidDF  41.0   F     NaN       NaN            NaN   
 3  840803046.0  thyroidDF  36.0   F     NaN       NaN            NaN   
 4  840803047.0  thyroidDF  32.0   F     NaN       NaN            NaN   
 
   iodine_deficiency diabetes obesity  ... fti tsh_measured t3_measured  \
 0               NaN      NaN     NaN  ... NaN            t           f   
 1               NaN      NaN     NaN  ... NaN            t           t   
 2               NaN      NaN     NaN  ... NaN            f           f   
 3               NaN      NaN     NaN  ... NaN            f           f   
 4               NaN      NaN     NaN  ... NaN            f           f   
 
   tt4_measured t4u_measured fti_measured  query_hyperthyroid  \
 0            f            f

In [2]:
df.columns

Index(['patient_id', 'source', 'age', 'sex', 'country', 'ethnicity',
       'family_history', 'iodine_deficiency', 'diabetes', 'obesity', 'lithium',
       'psych', 'pregnant', 'goitre', 'hypopituitary', 'tumor', 'nodule_size',
       'diagnosis', 'on_thyroxine', 'on_antithyroid_meds',
       'onantithyroidmedication', 'i131_treatment', 'tsh', 't3', 'tt4', 't4',
       't4u', 'fti', 'tsh_measured', 't3_measured', 'tt4_measured',
       't4u_measured', 'fti_measured', 'query_hyperthyroid',
       'query_hypothyroid', 'query_on_thyroxine', 'class'],
      dtype='object')

In [7]:
import pandas as pd
import numpy as np

HYPER_CODES = {"A","B","C","D"}

def has_hyper_code(x) -> bool:
    if pd.isna(x):
        return False
    s = str(x).strip()
    if s.lower() in ["low", "medium", "high"]:   # تاع cancer
        return False
    if s in ["-", ""]:
        return False

    # نقسم على | + , space
    parts = s.replace("+", "|").replace(",", "|").split("|")
    letters = set()
    for p in parts:
        p = p.strip()
        for ch in p:
            if "A" <= ch <= "Z":
                letters.add(ch)

    return len(HYPER_CODES.intersection(letters)) > 0


# thresholds (نقدرو نبدلوهم من بعد حسب distribution)
TSH_LOW = 0.4
T3_HIGH = 2.0
TT4_HIGH = 12.0

def biochemical_hyper(row) -> bool:
    # لازم tsh موجودة
    if pd.isna(row.get("tsh", np.nan)):
        return False

    if row["tsh"] >= TSH_LOW:
        return False

    high_flag = False
    if "t3" in row and not pd.isna(row.get("t3", np.nan)):
        high_flag = high_flag or (row["t3"] > T3_HIGH)

    if "tt4" in row and not pd.isna(row.get("tt4", np.nan)):
        high_flag = high_flag or (row["tt4"] > TT4_HIGH)

    return high_flag


def hybrid_target(row) -> int:
    # 1) diagnosis codes (أقوى)
    if has_hyper_code(row.get("class", np.nan)) or has_hyper_code(row.get("diagnosis", np.nan)):
        return 1

    # 2) biochemical rule
    if biochemical_hyper(row):
        return 1

    return 0

df["target_hyper_hybrid"] = df.apply(hybrid_target, axis=1)

df["target_hyper_hybrid"].value_counts(dropna=False)

target_hyper_hybrid
1    165884
0     55979
Name: count, dtype: int64

In [8]:
df[["tsh","t3","tt4"]].describe()

,tsh,t3,tt4
count,221021.000000,219259.000000,8730.000000
mean,4.941005,1.999396,108.302002
std,2.900993,0.860896,34.958622
min,0.120000,0.520000,16.000000
25%,2.390000,1.260000,87.000000
50%,4.900000,2.000000,104.000000
75%,7.460000,2.740000,126.000000
max,9.930000,3.470000,225.710000


In [9]:
df[["tsh","t3","tt4"]].isna().mean()

tsh    0.003795
t3     0.011737
tt4    0.960651
dtype: float64

In [10]:
from pathlib import Path

out_path = Path("../data/Unified Dataset/unified_dataset_with_hybrid_target.csv")
df.to_csv(out_path, index=False)
out_path

WindowsPath('../data/Unified Dataset/unified_dataset_with_hybrid_target.csv')

In [2]:
df["tsh"].describe()

NameError: name 'df' is not defined